# Imports

In [ ]:
import logging
import os

import pandas as pd
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

import core.config.config_ as ccocon
import helpers.dbg as hdbg
import helpers.env as henv
import helpers.printing as hprintin
import helpers.s3 as hs3
import im.data.universe as imdauni
import research.cc.statistics as rccsta
import research.cc.volume as rccvol
import im.ccxt.data.load.loader as imccdaloloa

import core.plotting as cplot
import numpy as np

# Config

In [ ]:
def get_cmtask323_config() -> ccocon.Config:
    """
    Get task323-specific config.
    """
    config = ccocon.Config()
    # Load parameters.
    config.add_subconfig("load")
    config["load"]["aws_profile"] = "am"
    config["load"]["data_dir"] = os.path.join(hs3.get_path(), "data")
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["data_type"] = "OHLCV"
    config["data"]["universe_version"] = "v0_3"
#        config["data"]["universe_version"] = "v0_1"
    config["data"]["vendor"] = "CCXT"
    # Column names.
    config.add_subconfig("column_names")
    config["column_names"]["volume"] = "volume"
    config["column_names"]["currency_pair"] = "currency_pair"
    config["column_names"]["exchange"] = "exchange_id"
    config["column_names"]["close"] = "close"
    return config


config = get_cmtask323_config()
print(config)

# Functions

In [ ]:
def compute_volatility_for_each_coin(data, freq):
    """
    Loads and transforms each (exchange-coin) dataframe to compute 18-period ema volatility.
    Parameters: DataFrame, resampling frequency
    """
    data["date"] = data.index
    new_df = data.groupby(["currency_pair", "exchange_id", pd.Grouper(key="date", freq=freq)])["close"].last()
    new_df = new_df.pct_change().transform(lambda x: x.ewm(span=18, adjust=False).std())
    new_df = new_df.reset_index()
    return new_df

def daily_close(data, freq):
    """
    Loads and transforms each (exchange-coin) dataframe to compute volatility for the whole period.
    Parameters: DataFrame, resampling frequency
    """
    data["date"] = data.index
    new_df = data.groupby(["currency_pair", "exchange_id", pd.Grouper(key="date", freq=freq)])["close"].last()
    new_df = new_df.reset_index()
    return new_df

def get_df_with_coin_price_volatility(data, display_plot):
    """
    Unifies volatility values for each coin and plot the graph.
    Parameters: DataFrame with computed volatility, boolean value to plot the graph
    """
    vix_df = data.groupby(["currency_pair", pd.Grouper(key="date", freq=frequency)])["close"].mean()
    vix_df = vix_df.to_frame()
    vix_df.columns = ["ema_volatility"]
    if display_plot:
        sns.set(rc={'figure.figsize':(15,8)})
        sns.lineplot(data=vix_df, x='date', y="ema_volatility", hue="currency_pair")
    return vix_df

def get_overall_returns_volatility(data, display_plot):
    """
    Unifies volatility values for each coin for the whole period and plot the barplot.
    Parameters: DataFrame with computed volatility, boolean value to plot the graph
    """
    close_df = daily_close.groupby(["currency_pair", pd.Grouper(key="date", freq=frequency)])["close"].mean()
    rets_df = close_df.groupby(["currency_pair"]).pct_change()
    std_df = rets_df.groupby(["currency_pair"]).std()
    if display_plot:
        cplot.plot_barplot(
            std_df.sort_values(ascending=False),
            title="Volatility per coin for the whole period (1-day basis, log-scaled)",
            figsize=[15, 7],
            yscale="log",
        )
    return std_df

def perform_adf_test(df_daily):
    """
    Performs ADF test to check the stationarity of volatility values
    Parameters: Daily DataFrame with computed volatility
    """
    final_result = []
    coin_list = df_daily.reset_index()["currency_pair"].unique()
    for coin in coin_list:
            result = pd.DataFrame()
            df = df_daily.loc[[f"{coin}"]]
            df[df["ema_volatility"].notna()]
            X = df[df["ema_volatility"].notna()]["ema_volatility"].values
            test_result = adfuller(X)
            result.loc[f"{coin}", "ADF Statistic"] = test_result[0]
            result.loc[f"{coin}", "p-value"] = test_result[1]
            final_result.append(result)
    final_result = pd.concat(final_result)
    final_result["is_unit_root_and_non-stationary (5% sign. level)"] = (final_result["p-value"]>0.05)
    return final_result

# Volatility Analysis

## 1 day

In [ ]:
frequency = "1D"
compute_daily_vix_ema = lambda data: compute_volatility_for_each_coin(data, freq=frequency)
daily_vix_ema = rccsta.compute_stats_for_universe(config, compute_daily_vix_ema)

In [ ]:
ema_df_daily = get_df_with_coin_price_volatility(daily_vix_ema, display_plot=True)
print(ema_df_daily)

## 5 min

In [ ]:
frequency = "5min"
compute_5min_vix_ema = lambda data: compute_volatility_for_each_coin(data, freq=frequency)
vix_ema_5min = rccsta.compute_stats_for_universe(config, compute_5min_vix_ema)

In [ ]:
ema_df_5min = get_df_with_coin_price_volatility(vix_ema_5min, display_plot=True)
print(ema_df_5min)

## Volatility for the whole period (1-day frequency)

In [ ]:
frequency = "1D"
compute_daily_close = lambda data: daily_close(data, freq=frequency)
daily_close = rccsta.compute_stats_for_universe(config, compute_daily_close)

In [ ]:
std_df = get_overall_returns_volatility(daily_close, display_plot=True)
print(std_df)

# Test for stationarity of volatility

In [ ]:
test_results = perform_adf_test(ema_df_daily)
test_results

After test results we see that __FIL/USDT__ volatility over 1-day is failed to pass the stationarity test. The graph below confirms the persistence of trend: seems like the coin was too volatile right after the listing and failed to keep the same levels during its trading lifetime.

In [ ]:
sns.lineplot(data=ema_df_daily.loc[["FIL/USDT"]].reset_index(), x='date', y='ema_volatility', hue="currency_pair")